In [ ]:
import os
import pandas as pd
path_to_incubations = "../../lab_work/sessions/incubations"


def load_incubation_data(path_to_incubations):
    dfs = []
    for file_name in os.listdir(path_to_incubations):
        path_to_file = os.path.join(path_to_incubations, file_name)
        print(path_to_file)
        dfs.append(pd.read_csv(path_to_file))

    result = pd.concat(dfs, ignore_index=True)

    return result
    # print(path_to_file)
    
def tidy_incubation_data(df):
    df['sample_name'] = df['sample_id']
    df['sample_id'] = df.apply(lambda row: set_sample_id(row), axis=1)
    df = df.drop(['sample_name'], axis=1)
    return df
    



def set_sample_id(row):
    sample_name = row['sample_name']
    if sample_name.startswith('40ml'):
        return f"40mL_{sample_name[-3:]}"
    else:
        return f"2mL_{sample_name}"
        
incubation_df = load_incubation_data(path_to_incubations=path_to_incubations)
incubation_df = tidy_incubation_data(incubation_df)
incubation_df

In [ ]:
import numpy as np
data_df = pd.read_csv('../../GCTCD_PDF_READER/GCPDFReader/csvs/tidied_1692821404.csv')

# np.unique(data_df['sample_id'])
data_df

In [ ]:
new_df = pd.merge(data_df, incubation_df, on='sample_id', how='left')
# new_df[new_df['Sample_Name']=='2ML_2_10']['sample_id']
new_df['ratio']

In [ ]:
np.unique(new_df[new_df['incubation_start_date'].isnull()]['sample_id'])

In [131]:

from datetime import datetime

def set_incubation_length(row):
    try:
        sample_date  = datetime.strptime(row['Sample_Date'], '%m/%d/%Y')
        incubation_start_date  = datetime.strptime(row['incubation_start_date'], '%m/%d/%y')
        return (sample_date - incubation_start_date).days

    except Exception as e:
        # print(e, row['Sample_Date'], row['incubation_start_date'])
        return None
    
def set_salt_to_biomass(row):
    if row['is_std']:
        return None
    if row['sample_id'] == 'DROP_ME':
        return None
    if row['ratio'] == "01:00":
        return "1:0"
    if row['ratio'] == "01:01":
        return "1:1"
    if row['ratio'] == "16:01":
        return "16:1"
    if row['ratio'] == "01:05":
        return "1:5"



def set_treatment_type(row):
    sample_id = row['sample_id']
    treatments = {"1": "Dry", "2": "Wet"}
    if sample_id.startswith('40mL'):
        return treatments[sample_id[5]]
    if sample_id.startswith('2mL'):
        return treatments[sample_id[4]]
    return None
        

new_df['incubation_length'] = new_df.apply(lambda row: set_incubation_length(row), axis=1)
new_df['salt_ratio'] = new_df.apply(lambda row: set_salt_to_biomass(row), axis=1)
new_df['str_ratio'] = new_df.apply(lambda row: str(row['ratio']), axis=1)
new_df['treatment'] = new_df.apply(lambda row: set_treatment_type(row), axis=1)


new_df

,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,...,notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added,incubation_length,salt_ratio,str_ratio,treatment
0,0,0,40ML1_10,8/16/2023,GCTCD,1,3.151,BP,494087.00000,188091.00000,...,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,1:0,01:00,Dry
1,1,1,40ML1_10,8/16/2023,GCTCD,2,4.693,PP,1595.22729,652.50348,...,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,1:0,01:00,Dry
2,2,2,40ML1_10,8/16/2023,GCTCD,3,5.270,BP,124.49847,48.04475,...,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,1:0,01:00,Dry
3,3,3,40ML1_10,8/16/2023,GCTCD,4,5.853,PV,188.87746,73.72960,...,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,1:0,01:00,Dry
4,4,4,40ML1_10,8/16/2023,GCTCD,5,5.961,VV,529.92957,109.93177,...,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,1:0,01:00,Dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,35,2.979pp m,8/14/2023,GCFID,1,2.387,MF,240.27603,31.48297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,nan,None
425,425,36,2.979pp m,8/14/2023,GCFID,2,2.495,FM,7.25621,4.14673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,nan,None
426,426,37,2.979pp m,8/14/2023,GCFID,3,2.567,FM,185.20935,15.03130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,nan,None
427,427,38,2.979pp m,8/14/2023,GCFID,1,2.385,BV,120.97746,26.07795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,nan,None


In [132]:
relevant_df  = new_df[(new_df['peak_compound'].notnull()) & (new_df['sample_id']!="DROP_ME")]
methane_df = relevant_df[(relevant_df['peak_compound'] == 'CH4') & (relevant_df['is_std'] == False)]
CO2_df = relevant_df[(relevant_df['peak_compound'] == 'CO2') & (relevant_df['is_std'] == False)]



In [95]:
CO2_df['calculated_conc'].min()
CO2_df['str_ratio'

221.05434968

In [153]:
from bokeh.models import Range1d
import pandas as pd
import numpy as np
import bokeh.io
from bokeh.models import  ColumnDataSource,Range1d, LabelSet, Label
from bokeh.palettes import Spectral6

from bokeh.io import export_png
from bokeh import plotting
from bokeh.models import LogColorMapper, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.transform import factor_cmap
from bokeh.transform import jitter
from bokeh. palettes import Inferno256
bokeh.io.output_notebook()





def create_chart(title, df):
    wet_df = ColumnDataSource(df[df["treatment"]=="Wet"])
    dry_df = ColumnDataSource(df[df["treatment"]=="Dry"])
    ratios = ["1:0", f"16:1", "1:1", "1:5"]

    TOOLTIPS = [
    ("sample_id", "@sample_id"),
    ("ratio", "@salt_ratio")
        ]
    exp_cmap = LinearColorMapper(palette=Inferno256, low=60, high=0)
    p = plotting.figure(
        title=title,
        # frame_height=500,
        # frame_width=800,
        x_axis_label = "Salt:Organic Matter",
        y_axis_label = "Calculated Conc (ppm)",
        x_range=ratios,
        y_axis_type="log",
        y_range=[10e0, 10e5],
        toolbar_location='above',
        tooltips = TOOLTIPS,
        sizing_mode="stretch_width"
    )
   
    # mapper = CategoricalColorMapper(palette="Viridis256", factors=str_ratio)

    p.triangle(
        source=wet_df,
        x = jitter("salt_ratio", width=0.3, range=p.x_range),
        # x = "calculated_conc",
        y = "calculated_conc",
        color={"field":"incubation_length", "transform":exp_cmap},
        size=10,
        legend_label="Wet"
    )
    p.plus(
        source=dry_df,
        x = jitter("salt_ratio", width=0.3, range=p.x_range),
        # x = "calculated_conc",
        y = "calculated_conc",
        color={"field":"incubation_length", "transform":exp_cmap},
        size=10,
        legend_label="Dry"
    )
    p.x_range.range_padding = 0
    # p.circle.glyph.fill_color = dict(field='salt_ratio', transform=mapper)

    # p.add_layout(bokeh.models.Legend(), "right")
    p.legend.click_policy = "hide"
    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    return p

hold = create_chart("CO2 conc", CO2_df)
bokeh.io.show(hold)



Loading BokehJS ...

In [ ]:
hold = incubation_length_v_conc("Methane conc", methane_df)
bokeh.io.show(hold)

Salt ratio v  conc 
- color by incubation length

In [155]:
from bokeh.models import Range1d
import pandas as pd
import numpy as np
import bokeh.io
from bokeh.models import  ColumnDataSource,Range1d, LabelSet, Label
from bokeh.palettes import Spectral6

from bokeh.io import export_png
from bokeh import plotting
from bokeh.models import LogColorMapper, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.transform import factor_cmap
from bokeh.transform import jitter

bokeh.io.output_notebook()





def salt_ratio_v_conc(title, df):
    df = ColumnDataSource(df)
    ratios = ["1:0", f"16:1", "1:1", "1:5"]

    TOOLTIPS = [
    ("sample_id", "@sample_id"),
    ("ratio", "@salt_ratio")
        ]
    exp_cmap = LinearColorMapper(palette=Inferno256, low=60, high=0)
    p = plotting.figure(
        title=title,
        # frame_height=500,
        # frame_width=800,
        x_axis_label = "Salt:Organic Matter",
        y_axis_label = "Calculated Conc (ppm)",
        x_range=ratios,
        y_axis_type="log",
        y_range=[10e0, 10e5],
        toolbar_location='above',
        tooltips = TOOLTIPS,
        sizing_mode="stretch_width"
    )
   
    # mapper = CategoricalColorMapper(palette="Viridis256", factors=str_ratio)

    p.circle(
        source=df,
        x = jitter("salt_ratio", width=0.6, range=p.x_range),
        # x = "calculated_conc",
        y = "calculated_conc",
        color={"field":"incubation_length", "transform":exp_cmap},
        size=10,
        )
    p.x_range.range_padding = 0
    # p.circle.glyph.fill_color = dict(field='salt_ratio', transform=mapper)

    # p.add_layout(bokeh.models.Legend(), "right")
    # p.legend.click_policy = "hide"
    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    return p

hold = salt_ratio_v_conc("CO2 conc v salt ratio", CO2_df)
bokeh.io.show(hold)



Loading BokehJS ...

In [129]:
hold = salt_ratio_v_conc("Methane conc v salt ratio", methane_df)
bokeh.io.show(hold)